<a href="https://colab.research.google.com/github/vaniamv/final-project-edit/blob/main/Streaming_test_vv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# autentication to gcloud with login

!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=m8PdlIeAZX3ogn5I5O0ViUHHUJ4cE5&prompt=consent&token_usage=remote&access_type=offline&code_challenge=ETwKvgRZc5s5KKKLZG2l2XMZyv8D7WZh6_pKqOFofoI&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrtCW-_fmrmdyMuLTZGWvyUZv0yFJVSmPzMBePnda91-Fs_EQauhm9PERrdJsYNo1A

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [3]:
# download connector and save it local

!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-04 19:35:26--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’

gcs-connector-hadoo 100%[===================>]  32.26M   204MB/s    in 0.2s    

2025-01-04 19:35:26 (204 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’ saved [33831577/33831577]



In [4]:
# import libraries

import os
from pyspark.sql import SparkSession

#spark session
spark = SparkSession.builder \
    .appName('GCS_Spark') \
    .config('spark.jars', '/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .getOrCreate()

# save credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/.config/application_default_credentials.json'

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", '/content/.config/application_default_credentials.json')


In [6]:
# Try reading a small file or a few records to avoid long processing times.
try:
  df = spark.read.format("json").load("gs://edit-de-project-streaming-data/carris-vehicles/vehicles_1735929391.json")
  df.show(5)  # Display the first 5 rows
  print("Connection successful!")
except Exception as e:
  print(f"Connection failed: {e}")

+-------+--------------------+--------------+--------+-----------------+-------+------------------+----------+--------+---------------------+------------+------------------+-------+----------+--------------------+
|bearing|            block_id|current_status|      id|              lat|line_id|               lon|pattern_id|route_id|schedule_relationship|    shift_id|             speed|stop_id| timestamp|             trip_id|
+-------+--------------------+--------------+--------+-----------------+-------+------------------+----------+--------+---------------------+------------+------------------+-------+----------+--------------------+
|     90|20250103-64410390...|   INCOMING_AT|44|12697|38.70689010620117|   4704|-8.962684631347656|  4704_0_1|  4704_0|            SCHEDULED|223440234560|13.333333333333332| 100398|1735929373|4704_0_1|1400|181...|
|     67|20250103-64410147...| IN_TRANSIT_TO|44|12655|38.68401336669922|   4701|-8.952688217163086|  4701_0_2|  4701_0|            SCHEDULED|223

In [7]:
from pyspark.sql.types import *

# create schema
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])


#readStreaming
stream = spark.readStream.format("json").schema(vehicle_schema).load("gs://edit-de-project-streaming-data/carris-vehicles")

In [20]:
stream.isStreaming

True

In [22]:


output_path="gs://edit-data-eng-project-group1/datalake/stream/vehicles_5"

# writeStreaming
query = (stream
        .writeStream
        #.queryName('table')
        .outputMode("append")
        #.foreachBatch(insert_vehicles)
        .option("path", output_path)
        .option('checkpointLocation', 'gs://edit-data-eng-project-group1/datalake/stream/vehicles_5/checkpoint')
        #.trigger(processingTime='5 seconds')
        .start()

        )

query.awaitTermination(60)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [23]:
query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [24]:
query.isActive

True

In [17]:
query.stop()

In [25]:
# Define the path to the Parquet files
parquet_path = "gs://edit-data-eng-project-group1/datalake/stream/vehicles_5"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()

# Perform some analysis or transformations (example: count rows)
row_count = parquet_df.count()
print(f"Number of rows: {row_count}")

# Example transformation: filter rows based on a condition
filtered_df = parquet_df.filter(parquet_df.speed > 50)

# Show the filtered data
filtered_df.show()

+-------+------------------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+---------------------------------+
|bearing|block_id                      |current_status|id      |lat      |line_id|lon      |pattern_id|route_id|schedule_relationship|shift_id    |speed    |stop_id|timestamp          |trip_id                          |
+-------+------------------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+---------------------------------+
|90     |20250103-64410390-223440234560|INCOMING_AT   |44|12697|38.70689 |4704   |-8.962685|4704_0_1  |4704_0  |SCHEDULED            |223440234560|13.333333|100398 |2025-01-03 18:36:13|4704_0_1|1400|1815_AEMLZ         |
|67     |20250103-64410147-223240234560|IN_TRANSIT_TO |44|12655|38.684013|4701   |-8.952688|4701_0_2  |4701_0  |SCHEDULE